In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Checking out the data i was given

In [11]:
from matplotlib import pyplot as plt
#import matplotlib.path as pat
from PIL import Image, ImageDraw, ImageOps
import json
#import timeit
import itertools
#from shapely.geometry import Point, Polygon
import random
import cv2 as cv
#import opendatasets as od
from pathlib import Path
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import torchvision


#od.download("https://www.kaggle.com/competitions/hubmap-organ-segmentation/data")
mpath = '../input/'
mpath_model = '../input/base-model'

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

plt.rcParams['figure.figsize'] = [15, 10]


df_train = pd.read_csv(mpath + '/hubmap-organ-segmentation/train.csv')
df_test = pd.read_csv(mpath + '/hubmap-organ-segmentation/test.csv')

In [12]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
import torch.nn.functional as F


class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        #self.in_ch = in_ch
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch,out_ch,3,1,1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch,out_ch,3,1,1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=False),
        )
        
    def forward(self,x):
        #print(x.shape)#self.in_ch)
        inter = self.conv(x) 
        #print(inter.shape)
        return inter

class UNET(nn.Module):
    def __init__(self,in_ch=3,out_ch=1,features=[64,128,256,512]):
        super(UNET,self).__init__()
        self.descend = nn.ModuleList()
        self.ascend = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        
        for f in features:
            self.descend.append(DoubleConv(in_ch,f))
            in_ch = f
            
        for f in reversed(features):
            self.ascend.append(nn.ConvTranspose2d(f*2,f,kernel_size=2,stride=2))
            self.ascend.append(DoubleConv(f*2,f))
            
        self.bottleneck = DoubleConv(features[-1],features[-1]*2)
        self.final_conv = nn.Conv2d(features[0],out_ch,kernel_size = 1)
        
    def forward(self,x):
        skip_connections = []
        #print(x.shape)
        for d in self.descend:
            #print(x.shape)
            x = d(x)
            #print(x.shape)
            skip_connections.append(x)
            #print(x.shape)
            x = self.pool(x)
            #print(x.shape)
        #print(2)
        x = self.bottleneck(x)
        
        skip_connections = skip_connections[::-1]
        #print(3)
        for idx in range(0,len(self.ascend),2):
            x = self.ascend[idx](x)
            skip_connection = skip_connections[idx//2]
            #print(4+idx-1)
            if x.shape != skip_connection.shape:
                x = TF.resize(x,size=skip_connection.shape[2:])
            
            concat_skip = torch.cat((skip_connection, x),dim=1)
            x = self.ascend[idx+1](concat_skip)
            
        return self.final_conv(x)
    
def test():
    x = torch.randn((3,1,161,161))
    model = UNET(in_ch=1,out_ch=1)
    preds = model(x)
    print(preds.shape)
    print(x.shape)
    assert preds.shape == x.shape
    
test()

In [13]:
#loss from the interwebs
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        #inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

Data

TODO:
1. account for crops of images
2. account for transforms of images(should be done on the fly vs preprocess (can slow down training) 

In [14]:
def get_crop_using_mask(im,w_slice=0,h_slice = 0,p_h = 0,p_w = 0):
    
    
    x1 = 0
    y1 = 0
    x2 = im.shape[1]
    y2 = im.shape[0]
    #pad if the image is smaller than the pad amount + crop
    h = w = False
    
    pad_front_w = pad_front_h = pad_back_w = pad_back_h = 0

    x1 = x1-w_slice
    if(x1<0):
        pad_front_w = -x1
        x1 = 0
        w = True
    y1 = y1-h_slice
    if(y1<0):
        pad_front_h = -y1
        y1 = 0
        h = True
    #print(x2)
    x2 = x2+(p_w-w_slice)
    #print(x2)
    if(x2>=im.shape[1]):
        pad_back_w = x2-im.shape[1]
        x2 = im.shape[1]
        w = True
    #print(y2)
    y2 = y2+(p_h-h_slice)
    #print(y2)
    if(y2>=im.shape[0]):
        pad_back_h = y2 - im.shape[0]
        y2 = im.shape[0]
        h = True
    #print(x1,x2,y1,y2)
    if(w or h):
        #print("Padding fucked:",pad_front_h,pad_back_h,pad_front_w,pad_back_w)
        if(len(im.shape) == 3):
            a = np.pad(im[y1:y2, x1:x2],pad_width=((pad_front_h,pad_back_h),(pad_front_w,pad_back_w),(0,0)),mode='reflect')
        else:
            a = np.pad(im[y1:y2, x1:x2],pad_width=((pad_front_h,pad_back_h),(pad_front_w,pad_back_w)),mode='reflect')
    else:
        a = im[y1:y2,x1:x2]

    return a

In [15]:
#testing crop
a = np.array(Image.open('../input/hubmap-organ-segmentation/train_images/62.tiff'))
print(a.shape)
im = get_crop_using_mask(a,w_slice=113,h_slice = 113,p_h = 227,p_w =227)
print(im.shape)
plt.imshow(im)
plt.show()

In [16]:
from torch.utils.data import Dataset

class HistoDataset(Dataset):
    #crops will be rectangular 
    def __init__(self,im_dir,mask_dir,labels,transform=None,crop_h = -1,trans_dir = './otsu'):
        self.im_dir = im_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.labels = pd.read_csv(labels)
        self.no_mirror = 4
        self.no_rot = 2
        self.crop_h = crop_h
        self.crop_w = crop_h
        self.rand = random.Random()
        rand.seed(2022)
        self.path_otsu = trans_dir
        initialise = False
        
        
        #need to add the pre transformed images this should be probs done befor training to cut down on costs
        
        if crop_h == -1:
            #we are using whole images
            series = [1]*len(self.labels)
            self.labels.insert(len(self.labels.columns),'len',series)
        else:
            #
            #example of image path: ./otsu250im and mask: ./otsu250mask and df is ./otsu250df.pickle
            #
            p_im = Path(self.path_otsu+str(crop_h)+"im")
            p_im.mkdir(parents=True, exist_ok=True)
            m_im = Path(self.path_otsu+str(crop_h)+"mask")
            m_im.mkdir(parents=True, exist_ok=True)
            with os.scandir(p_im) as it:
                initialise =  not any(it)
            #we will be using crops
            series = []
            padding_h = []
            padding_w = []
            slice_w = []
            slice_h = []
            mask_included = []
            crop_size = []
            #indexes = []
            #calc masks for tissue images and get the crop to calc amount of tiles(non overlaping with 0 padding)
            #pr = False
            if initialise:
                for ix, im in self.labels.iterrows():
                    #################!!!!!!!!!!!!!!!!!!!!!!!!!###################
                    #       need to also shift the mask for padding amount      #
                    #    either shift the mask points or just save bin masks    #
                    #           mask will be saved as bin numpy array           #
                    #          we also save the dataframe of the thing          #
                    #############################################################

                    #loading mask and image
                    im_path = os.path.join(self.im_dir, str(im.id) + ".tiff")
                    mask_path = os.path.join(self.mask_dir, str(im.id) + ".json") #to rabmo spremenit!!!! je JSON in ni actual slika
                    image = Image.open(im_path).convert("RGB")
                    masks = json.load(open(mask_path))
                    x,y = image.size
                    mask = np.array(maskPIL((y,x),masks,to_array =False).convert("L"),dtype=np.float32)
                    mask[mask==255.0] = 1

                    #
                    #getting the otsu_mask and padding for crop fit
                    m = otsu_mask_dilated(image,np.ones((10, 10), np.uint8),np.ones((300, 300), np.uint8))
                    m_c = get_crop_of_mask(m)
                    y_amount = m_c.shape[0] // self.crop_h #how many crops we can fit in without overlap, we want to get crops that include the tissue
                    p_h = self.crop_h - m_c.shape[0] % self.crop_h
                    x_amount = m_c.shape[1] // self.crop_w
                    p_w = self.crop_w - m_c.shape[1] % self.crop_w
                    im_len = self.no_mirror*self.no_rot*(y_amount + 1 if p_h>0 else 0 )*(x_amount + 1 if p_w>0 else 0 )
                    s_w = self.rand.randint(int(p_w//4),int(3*p_w//4))
                    s_h = self.rand.randint(int(p_h//4),int(3*p_h//4))
                    series.append(im_len)
                    padding_w.append(p_w)
                    slice_w.append(s_w)
                    padding_h.append(p_h)
                    slice_h.append(s_h)
                    #crop_size.append(m_c.shape)

                    #
                    #creating the crops
                    mask = get_crop_using_mask(m,mask,w_slice = s_w,h_slice = s_h,p_h = p_h,p_w = p_w)

                    #TODO check which crops hold the mask to have a well balanced train val split!
                    """for crop in range(im_len):
                        self.convert_index_to_transform(crop)
                        if np.sum(mask) > 0:"""

                    image = get_crop_using_mask(m,np.array(image),w_slice = s_w,h_slice = s_h,p_h = p_h,p_w = p_w)
                    np.save(os.path.join(p_im,str(im.id)),image)
                    np.save(os.path.join(m_im,str(im.id)),mask)

                self.labels.insert(len(self.labels.columns),'len',series)
                self.labels.insert(len(self.labels.columns),'padding_w',padding_w)
                self.labels.insert(len(self.labels.columns),'slice_w',slice_w)
                self.labels.insert(len(self.labels.columns),'padding_h',padding_h)
                self.labels.insert(len(self.labels.columns),'slice_h',slice_h)
                self.labels.to_pickle(path = "otsu"+str(crop_h) + "df.pkl")
                self.im_dir = p_im
                self.mask_dir = m_im

                #self.labels.insert(len(self.labels.columns),'crop_size',crop_size)
            else:
                #just read from the respective maps and the df
                self.labels = pd.read_pickle("otsu"+str(crop_h) + "df.pkl")
                self.im_dir = p_im
                self.mask_dir = m_im


            
        
    def __len__(self):
        #print(self.labels)
        return self.labels.loc[:,'len'].sum()


    #redoing getitem for the 3rd time because i don't know maths so we do only 0 or 90 rotation and 4 flips since flips are faster
    def __getitem__(self,index):
        l,index = self.convert_index(index)
        image = np.load(os.path.join(self.im_dir, str(l.id) + ".npy"))
        mask = np.load(os.path.join(self.mask_dir, str(l.id) + ".npy"))

        if self.transform is not None:
            rot,mirror,crop = self.convert_index_to_transform(index,l.len)
            print(rot,mirror,crop)
            #print(rot,mirror,crop)
            if mirror == 1:
                mask = mask[::-1,:]
                image = image[::-1,:,:]
            elif mirror == 2:
                mask = mask[:,::-1]
                image = image[:,::-1,:]
            else:
                mask = mask[::-1,::-1]
                image = image[::-1,::-1,:]
            #DONE replace this with own rotation if its faster
            image,mask = self.get_crop_rotated(image,mask,crop,rot,self.crop_h)
            #mask = np.rot90(mask,k=rot)
            #image = np.rot90(image,k=rot)
            #DONE this should be replaced to do the rotation and the crop extraciton inplace
            #y,x = self.get_crop_coord(crop,image)
            #im = np.copy(image[y:y+self.crop_h,x:x+self.crop_h])
            #mask = np.copy(mask[y:y+self.crop_h,x:x+self.crop_h])
            mask[mask == 255] = 1.0
            image = self.transform(np.copy(image)/255.0)
            mask = self.transform(np.copy(mask))


        return image,mask#,l

    #returns the actual crop of the mask and im
    def get_crop_rotated(self,im,mask,crop_idx,rot,crp_h):
        if rot == 0:
            y = int((crop_idx //(im.shape[1]//crp_h))*crp_h)
            x = int((crop_idx % (im.shape[1] // crp_h)) * crp_h)
            #print(y,x,"not flip")
            return im[y:y + crp_h, x:x + crp_h], mask[y:y + crp_h, x:x + crp_h]
        else:
            # rot1:(y,x) ->(im.shape[1]-x,y)
            y = int((crop_idx % (im.shape[0] // crp_h)) * crp_h)
            x = int((crop_idx // (im.shape[0] // crp_h)) * crp_h)
            #print(y,x)
            if im.shape[1]-1-x-crp_h<0:
                return np.swapaxes(im[y:y + crp_h, im.shape[1] - 1 - x::-1], 0, 1), np.swapaxes(mask[y:y + crp_h, im.shape[1] - 1 - x::-1], 0, 1)
            return np.swapaxes(im[y:y + crp_h, im.shape[1] - 1 - x:im.shape[1] - 1 - x - crp_h:-1], 0, 1), np.swapaxes(mask[y:y + crp_h, im.shape[1] - 1 - x:im.shape[1] - 1 - x - crp_h:-1], 0, 1)
   
    #image comes in already padded and cropped
    def get_crop_coord(self,crop,im):
        #first column index, then row index (columns are the inner loop and have higher freq)
        y_l,x_l = im.shape[:2]
        y = (crop // (x_l//self.crop_h))*self.crop_h
        x = (crop % (x_l//self.crop_h))*self.crop_h

        return int(y),int(x)
        
    
    def convert_index(self,index):
        suma = 0
        for ix, l in self.labels.iterrows():
            #print(l)
            if index <suma+l.len:
                return (l,index-suma)
            else:
                suma = suma + l.len

    #changed so number of rots and mirrors is not hardcoded
    def convert_index_to_transform(self,index,length):
        crops = length/(self.no_rot*self.no_mirror)
        rot = index //(self.no_rot*crops)
        mirror = (index %(self.no_rot*crops))//crops
        crop = (index %(self.no_rot*crops))%crops
        
        return rot,mirror,crop


    #speeding up the crops without rotating and flipping the whole array
    #flipping with indexing instead of np.flip is much faster around 4x faster
    #need to test rotation + crop
    #flip both axis and rotation is same as no flip and rotation and flip one axis is the same as flipping the other one -> only flip one axis and no flip


In [17]:
work_dir = "./"

In [29]:
from torch.utils.data import Dataset

class HistoDatasetTest(Dataset):
    #crops will be rectangular 
    def __init__(self,im_dir,labels,transform=None,crop_h = -1,work_dir="./"):
        self.im_dir = im_dir
        #self.mask_dir = mask_dir
        self.transform = transform
        self.labels = pd.read_csv(labels)
        self.no_mirror = 1
        self.no_rot = 1
        self.crop_h = crop_h
        self.crop_w = crop_h
        self.rand = random.Random()
        self.rand.seed(2022)
        self.trans_path = work_dir
        initialise = False
        
        
        #need to add the pre transformed images this should be probs done befor training to cut down on costs
        
        if crop_h == -1:
            #we are using whole images
            series = [1]*len(self.labels)
            self.labels.insert(len(self.labels.columns),'len',series)
        else:
            #
            #example of image path: ./otsu250im and mask: ./otsu250mask and df is ./otsu250df.pickle
            #
            p_im = Path(self.trans_path+str(crop_h)+"im")
            p_im.mkdir(parents=True, exist_ok=True)
            #m_im = Path(self.path_otsu+str(crop_h)+"mask")
            #m_im.mkdir(parents=True, exist_ok=True)
            with os.scandir(p_im) as it:
                initialise =  not any(it)
            #we will be using crops
            series = []
            padding_h = []
            padding_w = []
            slice_w = []
            slice_h = []
            mask_included = []
            crop_size = []
            h_amount = []
            w_amount = []
            #indexes = []
            #calc masks for tissue images and get the crop to calc amount of tiles(non overlaping with 0 padding)
            #pr = False

            for ix, im in self.labels.iterrows():
                #################!!!!!!!!!!!!!!!!!!!!!!!!!###################
                #       need to also shift the mask for padding amount      #
                #    either shift the mask points or just save bin masks    #
                #           mask will be saved as bin numpy array           #
                #          we also save the dataframe of the thing          #
                #############################################################

                #loading mask and image
                im_path = os.path.join(self.im_dir, str(im.id) + ".tiff")
                image = np.array(Image.open(im_path).convert("RGB"))
                x,y = image.shape[:2]


                #

                y_amount = image.shape[0] // self.crop_h #how many crops we can fit in without overlap, we want to get crops that include the tissue
                #how much to pad height
                p_h = self.crop_h - image.shape[0] % self.crop_h

                x_amount = image.shape[1] // self.crop_w
                #how much to pad width
                p_w = self.crop_w - image.shape[1] % self.crop_w
                im_len = self.no_mirror*self.no_rot*(y_amount + 1 if p_h>0 else 0 )*(x_amount + 1 if p_w>0 else 0 )

                s_w = p_w//2#self.rand.randint(int(p_w//4),int(3*p_w//4))
                s_h = p_h//2#self.rand.randint(int(p_h//4),int(3*p_h//4))
                series.append(im_len)
                padding_w.append(p_w)
                slice_w.append(s_w)
                padding_h.append(p_h)
                slice_h.append(s_h)
                h_amount.append(y_amount + 1 if p_h>0 else 0)
                w_amount.append(x_amount + 1 if p_w>0 else 0 )

                #creating the crops
                mask = get_crop_using_mask(image,w_slice = s_w,h_slice = s_h,p_h = p_h,p_w = p_w)

                #image = get_crop_using_mask(m,np.array(image),w_slice = s_w,h_slice = s_h,p_h = p_h,p_w = p_w)
                np.save(os.path.join(p_im,str(im.id)),mask)
                #np.save(os.path.join(m_im,str(im.id)),mask)

            self.labels.insert(len(self.labels.columns),'len',series)
            self.labels.insert(len(self.labels.columns),'padding_w',padding_w)
            self.labels.insert(len(self.labels.columns),'slice_w',slice_w)
            self.labels.insert(len(self.labels.columns),'padding_h',padding_h)
            self.labels.insert(len(self.labels.columns),'slice_h',slice_h)
            self.labels.insert(len(self.labels.columns),'h',h_amount)
            self.labels.insert(len(self.labels.columns),'w',w_amount)
            self.labels.to_pickle(path = work_dir+str(crop_h) + "df.pkl")
            self.im_dir = p_im
            #self.mask_dir = m_im

            #self.labels.insert(len(self.labels.columns),'crop_size',crop_size)





    def __len__(self):
        #print(self.labels)
        return self.labels.loc[:,'len'].sum()


    #redoing getitem for the 3rd time because i don't know maths so we do only 0 or 90 rotation and 4 flips since flips are faster
    def __getitem__(self,index):
        l,index = self.convert_index(index)
        image = np.load(os.path.join(self.im_dir, str(l.id) + ".npy"))
        h =  image.shape[0]
        w =  image.shape[1]
        #mask = np.load(os.path.join(self.mask_dir, str(l.id) + ".npy"))
        #print("image shape:",image.shape)
        #if self.transform is not None:
        crop = self.convert_index_to_transform(index,l.len)
        
        #DONE replace this with own rotation if its faster
        image = self.get_crop_rotated(image,crop,self.crop_h)
        #print("Pre:",image.shape)
        image = np.copy(image)
        #print("Mid:",image.shape)
        image = self.transform(image/255.0)
        #print("Post:",image.shape)
        #mask = self.transform(np.copy(mask))

        
        return image,l.id,l.padding_w,l.slice_w,l.padding_h,l.slice_h,l.h,l.w#,image

    #returns the actual crop of the mask and im
    def get_crop_rotated(self,im,crop_idx,crp_h):
        #print("Cropping:",im.shape," ",crop_idx,' ',crp_h,'\n')

        y = int((crop_idx //(im.shape[1]//crp_h))*crp_h)
        x = int((crop_idx % (im.shape[1] // crp_h)) * crp_h)
        #print(crop_idx,y,x,"not flip")
        return im[x:x + crp_h, y:y + crp_h,:]
        

    

    #image comes in already padded and cropped
    def get_crop_coord(self,crop,im):
        #first column index, then row index (columns are the inner loop and have higher freq)
        y_l,x_l = im.shape[:2]
        y = (crop // (x_l//self.crop_h))*self.crop_h
        x = (crop % (x_l//self.crop_h))*self.crop_h

        return int(y),int(x)
        
    
    def convert_index(self,index):
        suma = 0
        for ix, l in self.labels.iterrows():
            #print(l)
            if index <suma+l.len:
                return (l,index-suma)
            else:
                suma = suma + l.len

    #changed so number of rots and mirrors is not hardcoded
    def convert_index_to_transform(self,index,length):
        #crops = length/(self.no_rot*self.no_mirror)
        
        
        return index#crops


   

testing reading the test images

In [19]:
labels = "../input/hubmap-organ-segmentation/test.csv"
images = "../input/hubmap-organ-segmentation/test_images"
hist = HistoDatasetTest(images,labels,transform=None,crop_h = 250,work_dir="./")

In [20]:
"""for i in range(len(hist)):
    print(hist[i][1])
    plt.imshow(hist[i][0])
    plt.show()"""

In [21]:
c = Image.open(mpath+f'/hubmap-organ-segmentation/train_images/10044.tiff')
plt.figure(figsize=(20,20))
plt.imshow(c)
plt.show()

In [22]:
#print(len(hist))
"""
pri = True
n = 0
fig ,axs = plt.subplots(1,10,figsize=(10,10))
for i in range(220,400):
    #if pri:
        #print(hist[i][2].id)
    #print(hist[i][0].shape)
    if(n == 10):
        fig ,axs = plt.subplots(1,10,figsize=(10,10))
        n = 0
    axs[n].imshow(hist[i][0])
    #plt.imshow(hist[i][0])
    n = n+1
    #input()
"""

In [23]:
def convert_index_to_transform(index,length):
        crops = length/(4*4)
        rot = index //(4*crops)
        mirror = (index %(4*crops))//crops
        crop = (index %(4*crops))%crops
        
        return rot,mirror,crop
    
def convert_index(df,index):
        suma = 0
        for ix, l in df.iterrows():
            #print(l)
            if index <suma+l.len:
                return (l.id,index-suma)
            else:
                suma = suma + l.len
    
df = pd.DataFrame()
df.insert(0,'len',[15,12,33,14,3,4])
df.insert(0,'id',[0,1,2,3,4,5])
for i in range(df.loc[:,'len'].sum()):
    print(convert_index(df,i))

utils

In [30]:
from torch.utils.data import DataLoader,Subset
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler


def load_checkpoint(filename):
    print("LOADING MODEL")
    return torch.load(filename)



def get_loaders(
    ds,
    batch_size = 16,
    shuffle_ds = False,
    random_seed= 2022,
    num_workers = 4,
    pin_memory = True
):
    test_loader = DataLoader(
        ds,
        batch_size = batch_size,
        num_workers = num_workers,
        pin_memory = pin_memory,
    ) 
    return (ds,test_loader)    



In [32]:

import torch.optim as optim
import tqdm


lr = 1e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 1
N_WORKERS = 2
PIN_MEM = True
LOAD_MODEL = True
TEST_IM_DIR = "../input/hubmap-organ-segmentation/test_images"
model_stats = torch.load('../input/base-model/UNET33_BCELogits_1(whole_data).pth.tar',map_location=torch.device(DEVICE))

#def main(checkpoint = None):
train_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])


model = UNET(in_ch=3,out_ch=1).to(DEVICE)
model = model.float()
model.load_state_dict(model_stats['state_dict'])

labels = "../input/hubmap-organ-segmentation/test.csv"
images = "../input/hubmap-organ-segmentation/test_images"
hist = HistoDatasetTest(images,labels,transform=train_transforms,crop_h = 250,work_dir="./")

ds,test_loader = get_loaders(
    hist,
    #TRAIN_MASK_DIR,
    #train_transforms,
    #val_transforms,
    batch_size=BATCH_SIZE,
    num_workers = N_WORKERS,
    pin_memory = PIN_MEM
)
#this will be updated every time we are done with an image
crops = []
rle_encodings = []

Server side testing
#### Submission File

In order to reduce the submission file size, our metric uses run-length encoding on the pixel values. Instead of submitting an exhaustive list of indices for your segmentation, you will submit pairs of values that contain a start position and a run length. E.g. '1 3' implies starting at pixel 1 and running a total of 3 pixels (1,2,3).

Note that, at the time of encoding, the mask should be binary, meaning the masks for all objects in an image are joined into a single large mask. A value of 0 should indicate pixels that are not masked, and a value of 1 will indicate pixels that are masked.

The competition format requires a space delimited list of pairs. For example, '1 3 10 5' implies pixels 1,2,3,10,11,12,13,14 are to be included in the mask. The metric checks that the pairs are sorted, positive, and the decoded pixel values are not duplicated. The pixels are numbered from top to bottom, then left to right: 1 is pixel (1,1), 2 is pixel (2,1), etc.
The file should contain a header and have the following format:

In [41]:
#create mask to rle conversion:
#mask comes in cropped to the dimensions of the input
def bin_mask_to_rle(mask,id):
    rle_encoded = f"{id}, "
    #rows first from top to bottom
    for j in mask.shape[1]:
        starto = False
        start_pix = 1
        count = 0
        for i in mask.shape[0]:
            if mask[i,j] == 1:
                count = count +1
            else:
                if count>0:
                    rle_encoded.append(f"{start_pix - count} {count} ")
                    count = 0

            start_pix = start_pix + 1
    
    return rle_encoded

#stitiching the crops together
#crops go from up to down and from left to right
#padding is done only on the right and on the bottom(at max numbers on both axis)
#concatenation is wrongly done
def stitch_crops(h,w,crops,padding_h_befor,padding_w_befor,padding_h_after,padding_w_after):
    print(h,w)
    l = 0
    l_crops = []
    for crop in crops:
        if l == 0:
            l_crops.append(crop)
            #print(l_crops[0].shape)
        else:
            l_crops[-1] = np.concatenate((l_crops[-1],crop),axis=0)
        l = l +1
        #print(l)
        if l == w:
            #print("work?")
            l = 0
    print(len(l_crops))
    res = np.concatenate(l_crops,axis=1)
    return res[padding_h_befor:res.shape[0]-padding_h_after,padding_w_befor:res.shape[1]-padding_w_after]
    


In [42]:
#data = image,l.id,l.padding_w,l.slice_w,l.padding_h,l.slice_h,h,w

model.eval()
identi = None
prepad_w = 0
prepad_h = 0
postpad_w = 0
postpad_h = 0
h=0
w= 0
images = []
crops = []
count = 0
for data in test_loader:
    if identi is None or identi != data[1].item():
        if identi != data[1].item() and identi is not None:
            print("nope")
            images.append(stitch_crops(h,w,crops,prepad_h,prepad_w,postpad_h,postpad_w))
        indenti = data[1].item()
        prepad_w = data[3].item()
        prepad_h = data[5].item() 
        postpad_w = data[2].item() - data[3].item() 
        postpad_h = data[4].item() - data[5].item() 
        h = data[6].item()
        w = data[7].item()
        
    crops.append(data[0][0].detach().cpu().permute(1,2,0).numpy())
    if len(crops) > 1 and crops[-1].shape != crops[-2].shape:
        print("Prev shape:",crops[-2].shape)
        print("Cur shape:",crops[-1].shape)
        print(count)
    count = count +1
    
        
    
    
print(len(crops)   )
images.append(stitch_crops(h,w,crops,prepad_h,prepad_w,postpad_h,postpad_w))
#zadeve niso padane!!

In [43]:
plt.imshow(images[0])

In [38]:
a = np.array(Image.open('../input/hubmap-organ-segmentation/test_images/10078.tiff'))
print(a.shape)
#c = torchvision.transforms.ToTensor()
#im = get_crop_using_mask(a,w_slice=113,h_slice = 113,p_h = 227,p_w =227)
#print(im.shape)
plt.imshow(a)
plt.show()

TODO:
-create the test for client side DONE
-create the test for server side(conversion to poly and to rle)
-make a more balnced dataset (50/50 mask without mask)
-figure out if there is a better loss function than dice score
-do non labeled segmentation and run the seg on that? sort of like pretraining


We will try 2 concepts:
1. Predicting indiscriminant of organ with only one model
2. Training a model for each organ seperatly

option:
just rpedict the edge no need to predict the inside(use a thicker line for edge and just fill in the poly you get)

In [ ]:
torchvision.transforms.ToTensor(cxhxw)